# 말투 변환

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00


### import

In [ ]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
from tokenizers import Tokenizer
from typing import Dict, List, Optional
from torch.utils.data import Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import display
from typing import Dict

In [ ]:
data = pd.read_excel("/content/drive/MyDrive/text-transfer-data/말투변환_최종.xlsx")
data.head()

,formal,gentle
0,"‘비타민B7’ ‘비타민H’로도 불리는 비오틴은 모발과 피부, 손톱을 건강하게 유지하...","비타민 B7, 또는 비타민 H로 더 알려진 비오틴은 건강한 모발, 피부, 손톱 유지..."
1,"특히 머리카락을 구성하는 단백질인 케라틴 생성에 중요한 역할을 담당하고, 결핍 시 ...","'모발 비타민'은 머리카락을 구성하는 단백질인 케라틴 생성에 주요한 역할을 하며, ..."
2,탈모로 고민하는 사람들 사이에서 특히 이름난 영양소인 비오틴.,"비오틴은 이름난 영양소 중 하나로, 탈모로 고민하는 사람들에게 매우 효과적입니다."
3,비오틴의 역할과 효과 보는 섭취 팁을 소개한다.,"비오틴은 머리와 피부의 건강을 촉진하고 더 긴 두피와 머리카락, 손톱을 유지하기 위..."
4,WHAT: 이름도 생소한 영양소 ‘비오틴’#탈모 보조제의 단골 영양소‘비오틴’이라는...,'비오틴'이라는 단어는 탈모 샴푸나 탈모 보조제 패키지에서 가장 자주 볼 수 있는 ...


In [ ]:
data.describe()

,formal,gentle
count,15250,15250
unique,15236,15238
top,안녕하세요.,안녕하십니까
freq,8,5


In [ ]:
data.shape

(15250, 2)

- 15250개의 데이터 존재  

## 모델 불러오기

In [ ]:
model_name = "gogamza/kobart-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


## Data class 생성

In [ ]:
style_map = {
    'formal': '문어체',
    'gentle': '신사'
}

In [ ]:
class TextStyleTransferDataset(Dataset):
  def __init__(self,
               data: pd.DataFrame,
               tokenizer: Tokenizer
               ):
    self.data = data
    self.tokenizer = tokenizer

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    row = self.data.iloc[index, :].dropna().sample(2)
    text1 = row[0]
    text2 = row[1]
    target_style = row.index[1]
    target_style_name = style_map[target_style]

    encoder_text = f"{target_style_name} 말투로 변환:{text1}"
    decoder_text = f"{text2}{self.tokenizer.eos_token}"
    model_inputs = self.tokenizer(encoder_text, max_length=64, truncation=True)

    with self.tokenizer.as_target_tokenizer():
      labels = tokenizer(decoder_text, max_length=64, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    del model_inputs['token_type_ids']

    return model_inputs

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.1, random_state=42) # train, test 분리
print(len(train_data), len(test_data))

13725 1525


In [ ]:
train_dataset = TextStyleTransferDataset(
    train_data,
    tokenizer
)
test_dataset = TextStyleTransferDataset(
    test_data,
    tokenizer
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model
)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
model_path = "/content/drive/MyDrive/data/text-transfer-smilegate-bart-eos/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path,
    overwrite_output_dir=True,
    num_train_epochs=24,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_steps=500,
    save_steps=1000,
    warmup_steps=300,
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,2.029300,1.821229
1000,1.792600,1.752556
1500,1.625000,1.733328
2000,1.483300,1.727753
2500,1.432500,1.712313
3000,1.275200,1.743208
3500,1.227000,1.751908
4000,1.112300,1.757198
4500,1.066200,1.803071
5000,0.994900,1.789952


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3596: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

TrainOutput(global_step=20592, training_loss=0.6408213388113987, metrics={'train_runtime': 5136.7447, 'train_samples_per_second': 64.126, 'train_steps_per_second': 4.009, 'total_flos': 1.008682854746112e+16, 'train_loss': 0.6408213388113987, 'epoch': 24.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/data/text-transfer-smilegate-bart-eos/")

## 모델 학습 후 문장 변환

In [ ]:
from transformers import pipeline

nlg_pipeline = pipeline('text2text-generation',model=model_path, tokenizer=model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
def text_generate(pipe, text, target_style, num_return_sequences=5, max_length=60):
  target_style_name = style_map[target_style]
  text = f"{target_style_name} 말투로 변환:{text}"
  out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length)
  return [x['generated_text'] for x in out]

In [ ]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 17.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602628 sha256=182b870ae44a9f6a852cfaaf358f29eb911a68cbf2923311b53056070127114c
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


In [ ]:
from kiwipiepy import Kiwi
kiwi = Kiwi()

In [ ]:
src_text = "약의 약 효과를 제대로 보려면 음식도 가려 먹어야 하는데, 특히 알레르기 약, 고혈압 약, 고지혈증 치료제 등등은 약 효과를 떨어뜨리므로 음식도 가려 먹어야 한다. 해열진통제의 대표적인 성분인 아세트아미노펜(Acetaminophen)의 경우 부작용으로 간 손상과 위장관 출혈까지 일으킬 수 있으니 반드시 피해야 한다. 알레르기 약의 주성분은 '알부테롤', '클렌부테롤' 등의 성분이 카페인을 만나면 중추신경계를 자극해 흥분, 불안, 심박수 같은 부작용을 일으킬 수 있기 때문에 기관지 천식 환자나 만성 기관지염 환자는 가급적 커피를 포함한 카페인 음료를 피하는 것이 좋다. 항히스타민제를 이루고 있는 '펙소페나딘(Fexonadine)'은 알레르기 약 성분이 흡수되는 것을 방해할 수 있어 알레르기 약 성분이 흡수되는 것을 방해할 수 있다. 우유, 요거트 등 유제품을 섭취했다면 최소 1시간은 지난 후에 약을 복용해야 하며 고혈압약X육류고혈압 약 중 하나인 '베타차단제'는 심장 박동 수를 감소시키는 약이므로 소고기·돼지고기 등 붉은 육류 섭취는 피하는 것이 좋다. 혈액의 응고 작용을 지연시키는 항응고제인 와파린은 혈전 발생 위험이 큰 사람은 혈관이 막히는 것을 방지하는 항응고제이며 특히 고혈압 환자의 경우 특별한 주의가 필요하다. 시금치· 상추· 양배추·브로콜리 등의 녹색 채소·콩 등이 함유된 자몽 주스를 섭취할 경우 약에 미치는 효과가 72시간까지 지속되므로 72시간 간격 두고 섭취할 것을 권장한다. "

In [ ]:
## new
src_text="닮은 듯 다른 건망증과 치매 구별법을 알아보자. 뇌에 정보를 저장하는 기능에는 문제가 없지만 기억해 내는 속도가 느리거나 순간적으로 바로 기억하지 못하는 건망증의 가장 큰 특징은 중요하지 않은 사소한 일들은 잊지만, 중요한 약속이나 큰 사건들은 잊지 않고 기억할 수 있다는 것이다. 기억력이 나빠진 것조차 인지 못 하는 치매는 뇌에 비정상 단백질이 쌓이는 '퇴행성 뇌 질환', 뇌졸중이나 뇌출혈과 같은 '뇌혈관질환', 외상으로 인한 '뇌 손상' 또는 비타민B 등 신경계의 필수 영양소 결핍으로 인한 '뇌 기능 저하'가 원인이다. 기억력 저장 과정에서 기억력이 나빠진 것조차 인지하지 못하는 건망증과 달리, 치매는 힌트를 줘도 전혀 떠올리지 못하는 건망증 구별법 뇌가 기억을 저장하는 과정을 살펴보면 나의 이 ‘깜빡깜빡’하는 증상이 치매 증상인지 건망증 증상인지 어느 정도 구별된다. 치매는 뇌세포 손상으로 기억을 저장하는 과정이 망가져 문제가 생기는 경우로 전두엽 기능이 떨어지면서 인출에 장애가 생기는 현상인 반면, 건망증은 신경 조직 손상으로 인해 기억을 저장하는 과정 자체가 망가져 문제가 생기는 경우로 치매와 신경 조직 손상으로 인해 기억을 저장하는 과정 자체가 망가져 문제가 생기는 경우로 구분할 수 있다. 외출 준비를 하면서 여러 가지 일을 동시에 처리할 때 일어나는 건망증인지, 아니면 주차선 밖에 버젓이 주차를 해뒀다면 치매 증상이 될 수 있다. 수년간 정기적으로 같은 날짜에 병원을 방문해 진료받고 있는데, 그 날짜를 착각하는 것 역시 치매 증상 중 하나인 지남력 장애일 가능성이 높으니 주의하자. 치매 이전에 전 단계에 해당하며, 동일 연령대에 비해 기억력 등 인지 기능이 감소한 상태를 일컫는 경도인지장애는 치매의 전 단계에 해당하며, 치매의 전 단계일 수도 있어 주의해야 한다. 기억력에는 별다른 문제가 없더라도 다른 인지 기능이 약해졌거나, 성격이 갑자기 바뀌거나, 사고와 행동이 느려지는 증상도 경도인지장애를 알리는 신호일 수 있으므로 신경과 전문의를 찾아 정밀검사를 받아볼 것. 기억력 강화 훈련을 위해서는 메모지를 보지 말고 목록을 외워 구매하는 방법을 써보길 바란다. 치매 초기 초기 환자들은 소설을 완독하지 못하는 경우가 많다는 연구 결과도 있어 단편소설부터 읽는 습관을 들여야 한다. 치매 상담점수가 높을수록 주관적 기억감퇴가 심한 것을 의미하므로 보건소의 치매안심센터에서 치매 조기검진을 받을 것을 권장한다. 치매 조기검진을 받아 볼 것! 정부는 치매 조기검진 사업을 실시하고 있다. 보건소의 치매안심센터에서는 만 60세 이상 지역주민에게 치매선별검사를 무료로 제공하고, 검사 결과에 이상이 있을 시 치매 진단· 감별검사까지 지원하고 있으니 조금이라도 치매가 의심된다면, 반드시 보건소 방문을 권한다"

In [ ]:
sentences = kiwi.split_into_sents(src_text)
transfer = []

for i in sentences:
  style = "gentle"
  print("입력 문장 :", i[0])
  print("after : ", text_generate(nlg_pipeline, i[0], style, num_return_sequences=1, max_length=1000)[0])
  transfer.append(text_generate(nlg_pipeline, i[0], style, num_return_sequences=1, max_length=1000)[0])
  print('\n')

입력 문장 : 닮은 듯 다른 건망증과 치매 구별법을 알아보자.
after :  어떤 건망증과 치매 구별법을 알아볼까요?


입력 문장 : 뇌에 정보를 저장하는 기능에는 문제가 없지만 기억해 내는 속도가 느리거나 순간적으로 바로 기억하지 못하는 건망증의 가장 큰 특징은 중요하지 않은 사소한 일들은 잊지만, 중요한 약속이나 큰 사건들은 잊지 않고 기억할 수 있다는 것이다.
after :  건망증은 뇌에 정보를 저장하는 기능은 문제 없지만, 속도가 느리거나 순간적으로 기억하지 못하는 것이 가장 큰 특징입니다. 중요한 사소한 일들은 잊을 수 있지만, 중요한 약속이나 큰 사건들은 잊지 않고 기억할 수 있습니다.


입력 문장 : 기억력이 나빠진 것조차 인지 못 하는 치매는 뇌에 비정상 단백질이 쌓이는 '퇴행성 뇌 질환', 뇌졸중이나 뇌출혈과 같은 '뇌혈관질환', 외상으로 인한 '뇌 손상' 또는 비타민B 등 신경계의 필수 영양소 결핍으로 인한 '뇌 기능 저하'가 원인이다.
after :  치매는 뇌 기능 저하를 초래하는 '퇴행성 뇌 질환', 뇌졸중이나 뇌출혈과 같은 '뇌혈관질환', 뇌 손상이나 비타민B 부족으로 인한 '뇌 기능 저하' 등의 증상이 있습니다.


입력 문장 : 기억력 저장 과정에서 기억력이 나빠진 것조차 인지하지 못하는 건망증과 달리, 치매는 힌트를 줘도 전혀 떠올리지 못하는 건망증 구별법 뇌가 기억을 저장하는 과정을 살펴보면 나의 이 ‘깜빡깜빡’하는 증상이 치매 증상인지 건망증 증상인지 어느 정도 구별된다.
after :  치매는 기억력 저장 과정에서 기억력이 나빠진 것조차 인지하지 못하는 것이므로, 힌트를 줘도 전혀 떠올리지 못하는 것을 구별할 수 있습니다.


입력 문장 : 치매는 뇌세포 손상으로 기억을 저장하는 과정이 망가져 문제가 생기는 경우로 전두엽 기능이 떨어지면서 인출에 장애가 생기는 현상인 반면, 건망증은 신경 조직 손상으로 인해 기억을 저장하는 과정 자체가 망가져 문제가 생기는 경우로 치매와 신경 조직 손상으로 인해 기억을 저장하는 과정 

In [ ]:
transfer

['어떤 건망증과 치매 구별법을 알아볼까요?',
 '건망증은 뇌에 정보를 저장하는 기능은 문제 없지만, 속도가 느리거나 순간적으로 기억하지 못하는 것이 가장 큰 특징입니다. 중요한 사소한 일들은 잊을 수 있지만, 중요한 약속이나 큰 사건들은 잊지 않고 기억할 수 있습니다.',
 "치매는 뇌 기능 저하를 초래하는 '퇴행성 뇌 질환', 뇌졸중이나 뇌출혈과 같은 '뇌혈관질환', 뇌 손상이나 비타민B 부족으로 인한 '뇌 기능 저하' 등의 증상이 있습니다.",
 '치매는 기억력 저장 과정에서 기억력이 나빠진 것조차 인지하지 못하는 것이므로, 힌트를 줘도 전혀 떠올리지 못하는 것을 구별할 수 있습니다.',
 '치매는 기억을 저장하는 과정이 망가져 문제가 생기는 현상으로, 전두엽의 기능이 저하되며 인출에 문제가 생기는 건망증과 신경 조직 손상으로 인해 기억을 저장하는 과정 자체가 망가져 문제가 생기는 건망증으로 구분할 수 있습니다.',
 '만약 외출 준비를 하면서 여러 가지 일을 동시에 처리할 때 건망증인지 아니면 주차선 밖에 버젓이 주차를 해둔 것인지 모른다면, 이는 치매 증상이 될 수 있습니다.',
 '정기적으로 같은 시간에 병원을 방문하는데, 날짜를 변경하는 것은 지남력 장애의 증상 중 하나일 수 있으므로 주의해야 합니다.',
 '치매 전 단계에 해당하는 경도인지장애는 기억력 등 인지 기능이 동일 연령대에 비해 감소하는 상태를 말합니다. 그러나 치매의 전 단계에 해당하는 경도인지장애는 주의해야 합니다.',
 '만약 기억력이 크게 문제가 없더라도, 성격 변화가 갑작스럽게 일어나거나 사고와 행동이 느려지는 증상은 경도인지장애를 알리는 신호일 수 있으므로 신경과 전문의를 찾아 정밀검사를 받아보세요.',
 '기억력 강화 훈련을 위해 메모지를 보지 말고 목록 외워 구매하는 방법을 사용해보세요.',
 '치매 초기 초기 환자들은 소설을 완독하지 못하는 경우가 많다고 알려져 있으며, 단편소설부터 읽는 습관을 들여야 한다는 연구 결과가 있습니다.',
 '치매 상담점수가 높을수록 치매

## CSV 저장

In [ ]:
df_transfer = pd.DataFrame(transfer,columns=['transfer'])
df_transfer.to_csv('summaryNspeechtrans.csv',index=False)